In [ ]:
import pandas as pd
import numpy as np
from copy import deepcopy
import json
import matplotlib.pyplot as plt
import datetime
import shutil
import os
import re

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
import keras
import pickle

import requests
from google.cloud import storage
from bs4 import BeautifulSoup

In [ ]:
# Make sure the following environment variable is set! 
creds_path = os.environ['GOOGLE_APPLICATION_CREDENTIALS_PATH']
gcs_client = storage.Client.from_service_account_json(creds_path)
bucket = gcs_client.get_bucket('covid-19-forecaster-data')

In [ ]:
summary_char_limit = 130

In [ ]:
# Params
begin_date_str = '1/27/20' # Make sure it's a monday, and the day before exists in the data
begin_date = datetime.datetime.strptime(begin_date_str, "%m/%d/%y")

today = datetime.date.today()
today_date = f'{today.month}/{today.day}/{today.year-2000}'
today_date

In [ ]:
response_names = {'UK': 'GBR', 'US': 'USA'}

In [ ]:
# Create output folder
week_begin = today - datetime.timedelta(today.weekday())
week_end = week_begin + datetime.timedelta(days=6)
cur_week_name = f'{week_begin.month}-{week_begin.day}-{week_begin.year-2000}-to-{week_end.month}-{week_end.day}-{week_end.year-2000}'
preds_folder = f'./preds/weekly/{cur_week_name}'
output_folder = f'./output/weekly/{cur_week_name}'
print(f'Current identifier: *{cur_week_name}*')

# Read existing data

In [ ]:
# Load responses
responses = {}
for name in response_names:
    print(name)
    with open(f'{preds_folder}/response-{name}.json') as json_file: 
        responses[name] = json.load(json_file)

In [ ]:
joint_keys = set(responses['UK'].keys()) & set(responses['US'].keys())
len(joint_keys)

## Create countries response

In [ ]:
country_response = {}
country_keys = [x for x in joint_keys if len(x) == 3]
joint_fields = ['risk_preds']
# country_fields = ['category', 'advice_url', 'summary', 'category_preds', 'confidence_preds']
country_fields = ['category', 'advice_url', 'summary', 'category_preds']
for key in country_keys:
    country_response[key] = {}
    
    # Get common fields first
    for field in joint_fields:
        if field in responses['UK'][key]:
            country_response[key][field] = responses['UK'][key][field]

    for field in country_fields:
        if field not in responses['UK'][key]:
            continue
        country_response[key][field] = {}
        for name in response_names:
            if field == 'summary':
                country_response[key][field][response_names[name]] = responses[name][key][field][:summary_char_limit]
            else:
                country_response[key][field][response_names[name]] = responses[name][key][field]

## Create Regional response

In [ ]:
region_response = []
region_keys = [x for x in joint_keys if len(x) > 3]
joint_fields = ['risk_preds']
region_fields = ['lat', 'lon', 'ccode', 'graph_name']
for key in region_keys:
    new_response = {'name': key[:-4], 'country': responses['UK'][key]['country']}
    
    # Get common fields first
    for field in joint_fields:
        if field in responses['UK'][key]:
            new_response[field] = responses['UK'][key][field]
    
    # Get region fields (they happen to be the shared ones)
    for field in region_fields:
        if field in responses['UK'][key]:
            new_response[field] = responses['UK'][key][field]
    region_response.append(new_response)

## Upload JSONs

In [ ]:
last_updated_txt = datetime.datetime.utcnow().strftime("last updated: %Y-%m-%d at %H:%M:%S (UTC)")
last_updated_txt

In [ ]:
main_response = {'last_updated': last_updated_txt, 'data': country_response}

### First, upload results to this week's folders

In [ ]:
# Upload country json to current week's folder
with open(f'{preds_folder}/countries.json', 'w') as json_file:
    json.dump(main_response, json_file, indent='\t')
blob = bucket.blob(f"preds/weekly/{cur_week_name}/countries.json")
blob.upload_from_filename(f'{preds_folder}/countries.json')

# Upload region json to current week's folder
with open(f'{preds_folder}/regions.json', 'w') as json_file:
    json.dump(region_response, json_file, indent='\t')
blob = bucket.blob(f"preds/weekly/{cur_week_name}/regions.json")
blob.upload_from_filename(f'{preds_folder}/regions.json')

In [ ]:
# Upload the output folder of this week to azure as well.

# Delete this week's output if it's up there
blobs = bucket.list_blobs(prefix=f"output/weekly/{cur_week_name}")
for blob in blobs:
    blob.delete()
    
# Upload the files
for path, subdirs, files in os.walk(output_folder):
    path = path.replace("\\","/")
    directory_name = path.replace(output_folder, f"output/weekly/{cur_week_name}")
    for file in files:
        blob = bucket.blob(directory_name+'/'+file)
        blob.upload_from_filename(os.path.join(path, file))

### Finally upload the response jsons to the public "current" folder

In [ ]:
# Upload country json to public folder
with open(f'{preds_folder}/countries.json', 'w') as json_file:
    json.dump(main_response, json_file, indent='\t')
blob = bucket.blob(f"preds/weekly/current/countries.json")
blob.upload_from_filename(f'{preds_folder}/countries.json')
blob.make_public() # BE CAREFUL

# Upload region json to public folder
with open(f'{preds_folder}/regions.json', 'w') as json_file:
    json.dump(region_response, json_file, indent='\t')
blob = bucket.blob(f"preds/weekly/current/regions.json")
blob.upload_from_filename(f'{preds_folder}/regions.json')
blob.make_public() # BE CAREFUL

In [ ]:
main_response

In [ ]:
region_response

In [ ]:
f'{preds_folder}/countries.json'

In [ ]:
output_folder